# Foundations of CSS Final Group Project: *Question Goes Here*

TODO: Add link to research poster and report

## Prerequisits

### Install Dependencies

In [ ]:
!pip install pyreadstat pandas numpy matplotlib

### Import Libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import zipfile
import shutil

### Decompress Data

In [4]:
with zipfile.ZipFile("data.zip", "r") as zip_ref:
    zip_ref.extractall("data")
 
# For data changes, re-zip data to zip file with python:
# shutil.make_archive("data", 'zip', "data")
# or in the shell with command `zip data.zip data/ -9 -r` (linux)


### Load Data


International Social Survey Programme: Environment I-IV Cumulation. \
ISSP Research Group (2024) \
GESIS, Cologne. \
ZA8793 Data file Version 1.0.0 \
https://doi.org/10.4232/1.14332

See [terms of use](https://www.gesis.org/fileadmin/upload/dienstleistung/daten/umfragedaten/_bgordnung_bestellen/2023-06-30_Nutzungsbedingungen.pdf) (category A) for more information.

We do this in analogy to the paper https://search.gesis.org/research_data/ZA7650?doi=10.4232/1.13921 (Long-run trends in partisan polarization of climate policy-relevant attitudes across countries)

In [ ]:
attitudes_df = pd.read_stata("data/ZA8793_v1-0-0_survey.dta")
attitudes_df


---

TODO: Fossil production data

---

TODO: Fossil dependence data

---

TODO: personal vs public action data


In [ ]:
gas_data = pd.read_csv("data/Gas.csv", header=0 )
petroleum_data = pd.read_csv("data/patrolium production.csv", header = 1)
coal_data = pd.read_csv("data/Coal.csv", header = 0)

In [7]:
def clean_data(data_set):
    # Remove rows where 'API' is 0 to filtred out the rows that are not countires and countires that does not have any data
    data_set = data_set[data_set['API'].fillna(0) != 0]
    data_set = data_set[data_set['API'] != 0]
    
    #Group by 'Country'
    country_combined_data = data_set.groupby('Country').sum()

    for column in country_combined_data.columns:
        if column not in ['Country', 'API']:
            country_combined_data[column] = country_combined_data[column].replace({',': '', ' ': ''}, regex=True)
            country_combined_data[column] = country_combined_data[column].astype(str)
            country_combined_data[column] = country_combined_data[column].str.extract('(\d+(\.\d+)?)')[0]
            country_combined_data[column] = pd.to_numeric(country_combined_data[column])

    #country_combined_data = country_combined_data.astype(float)
    # Reset index
    country_combined_data = country_combined_data.reset_index()
    
    # Return
    return country_combined_data


cleaned_petroleum_data = clean_data(data_set = petroleum_data)
cleaned_gas_data = clean_data(data_set = gas_data)
cleaned_coal_data = clean_data(data_set= coal_data)

In [ ]:
def total_energy_calculation(data_set_1, data_set_2, data_set_3):
    # Concatenate the three datasets
    dataset = pd.concat([data_set_1, data_set_2, data_set_3])
    combined_data_set = dataset.groupby('Country').sum()
    combined_data_set = combined_data_set.reset_index()
    #print("Before conversion:", combined_data_set['Country'].dtype)
    combined_data_set['Country'] = combined_data_set['Country'].str.strip().map(str)
    #print("After conversion:", combined_data_set['Country'].dtype)
    #print(combined_data_set['Country'].unique())

    
    return combined_data_set


total_energy_production = total_energy_calculation(cleaned_petroleum_data, cleaned_gas_data, cleaned_coal_data)
total_energy_production

### Merge Datasets

In [9]:
# TODO
